# Day 4: Data Preparation & Feature Engineering - SOLUTIONS

**Duration:** 90 minutes  
**Dataset:** Titanic Passenger Data

## Learning Objectives
- Understand data quality properties and the knowledge hierarchy
- Distinguish structured vs unstructured data
- Handle missing data using different strategies
- Detect and handle outliers
- Apply normalization techniques
- Perform categorical encoding
- Create new features through feature engineering

---

## Part 1: Setup and Data Loading (5 mins)

In [ ]:
# Import libraries
import pandas as pd
import numpy as np
import plotly.express as px
import plotly.graph_objects as go
import seaborn as sns
from scipy import stats
from sklearn.preprocessing import MinMaxScaler, StandardScaler
from sklearn.ensemble import IsolationForest

print("✓ Libraries imported successfully!")

In [ ]:
# Load Titanic dataset
df = sns.load_dataset('titanic')
print(f"Dataset loaded: {df.shape[0]} rows, {df.shape[1]} columns")
df.head()

---
## Part 2: Datenqualität & Wissenspyramide (10 mins)

### Die Wissenspyramide
- **Daten:** Rohe Fakten (z.B. "Alter: 22")
- **Information:** Verarbeitete Daten (z.B. "Durchschnittsalter ist 29.7")
- **Wissen:** Muster verstehen (z.B. "Jüngere Passagiere hatten bessere Überlebensraten")
- **Weisheit:** Wissen anwenden (z.B. "Kinder in Notfällen priorisieren")

### 🎯 Vorlesungsübung 2.1: Fehlende Werte identifizieren - SOLUTION (5 mins)

In [ ]:
# Prüfe auf fehlende Werte
missing_values = df.isnull().sum()
print("📊 Fehlende Werte pro Spalte:")
print(missing_values[missing_values > 0])
print(f"\n⚠️ Gesamt fehlende Werte: {missing_values.sum()}")

In [ ]:
# Visualisiere fehlende Daten als Heatmap
missing_data = df.isnull()
fig = px.imshow(missing_data.T, 
                labels=dict(x="Passagier", y="Feature", color="Fehlt"),
                title="Heatmap fehlender Daten (Weiß = fehlt)")
fig.show()

---
## Part 3: Fehlende Daten behandeln (15 mins)

### Strategien zum Umgang mit fehlenden Daten
1. **Löschen** - Zeilen/Spalten entfernen
2. **Imputation** - Fehlende Werte ersetzen (Mean, Median, Mode)
3. **Prediction** - ML-Modell zur Vorhersage fehlender Werte

### 🎯 Vorlesungsübung 3.1: Mean vs. Median Imputation - SOLUTION (7 mins)

In [ ]:
# Strategie 1: Mean Imputation (Mittelwert)
df['age_mean_imputed'] = df['age'].fillna(df['age'].mean())

print(f"📊 Originales Durchschnittsalter: {df['age'].mean():.2f}")
print(f"📊 Nach Mean Imputation: {df['age_mean_imputed'].mean():.2f}")

In [ ]:
# Strategie 2: Median Imputation (Median)
df['age_median_imputed'] = df['age'].fillna(df['age'].median())

print(f"📊 Originaler Median: {df['age'].median():.2f}")
print(f"📊 Nach Median Imputation: {df['age_median_imputed'].median():.2f}")
print(f"\n💡 Median ist robuster gegenüber Ausreißern!")

In [ ]:
# Vergleiche die Verteilungen
fig = go.Figure()
fig.add_trace(go.Histogram(x=df['age'].dropna(), name='Original', opacity=0.6, nbinsx=30))
fig.add_trace(go.Histogram(x=df['age_mean_imputed'], name='Mean Imputation', opacity=0.6, nbinsx=30))
fig.add_trace(go.Histogram(x=df['age_median_imputed'], name='Median Imputation', opacity=0.6, nbinsx=30))
fig.update_layout(title='Vergleich der Imputation-Strategien',
                  xaxis_title='Alter',
                  yaxis_title='Anzahl',
                  barmode='overlay')
fig.show()

---
## Part 4: Ausreißer erkennen (10 mins)

### 🎯 Vorlesungsübung 4.1: Box Plot zur Ausreißererkennung - SOLUTION (5 mins)

In [ ]:
# Erstelle einen Box Plot für die Fahrpreise
fig = px.box(df, y='fare', 
             title='Fahrpreis-Verteilung mit Ausreißern',
             labels={'fare': 'Fahrpreis (£)'})
fig.show()

print("💡 Die Punkte außerhalb der Box sind potenzielle Ausreißer!")

---
## Part 5: Normalisierung (10 mins)

### Warum Normalisierung?
Viele ML-Algorithmen arbeiten besser, wenn Features auf ähnlichen Skalen sind.

**Zwei Hauptmethoden:**
1. **Min-Max Scaling:** Skaliert auf [0, 1]
2. **Standardization (Z-Score):** Mittelwert=0, Standardabweichung=1

### 🎯 Vorlesungsübung 5.1: Normalisierung anwenden - SOLUTION (5 mins)

In [ ]:
# Min-Max Scaling: Skaliere 'age' auf [0, 1]
scaler_minmax = MinMaxScaler()
df['age_minmax'] = scaler_minmax.fit_transform(df[['age_median_imputed']])

print(f"📊 Original Alter - Min: {df['age_median_imputed'].min():.2f}, Max: {df['age_median_imputed'].max():.2f}")
print(f"📊 Nach Min-Max - Min: {df['age_minmax'].min():.2f}, Max: {df['age_minmax'].max():.2f}")

In [ ]:
# Standardization (Z-Score): Mittelwert=0, Std=1
scaler_standard = StandardScaler()
df['age_standardized'] = scaler_standard.fit_transform(df[['age_median_imputed']])

print(f"📊 Nach Standardization - Mean: {df['age_standardized'].mean():.2f}, Std: {df['age_standardized'].std():.2f}")

---
## Part 6: Kategorische Kodierung (10 mins)

### Warum Kodierung?
ML-Modelle benötigen numerische Eingaben! Wir müssen kategorische Variablen umwandeln.

### 🎯 Vorlesungsübung 6.1: Binary Encoding - Geschlecht - SOLUTION (5 mins)

In [ ]:
# Kodiere 'sex' als binären Wert: male=1, female=0
df['sex_encoded'] = df['sex'].map({'male': 1, 'female': 0})

print("📊 Geschlechts-Kodierung:")
print(df[['sex', 'sex_encoded']].drop_duplicates())

---
## Part 7: Feature Engineering (15 mins)

**Wichtig fürs Projekt!** Feature Engineering = neue Features aus bestehenden Daten erstellen.

### 🎯 Vorlesungsübung 7.1: Family Size Feature - SOLUTION (7 mins)

In [ ]:
# Erstelle 'family_size' Feature
# family_size = SibSp (Geschwister/Partner) + Parch (Eltern/Kinder) + 1 (Person selbst)

df['family_size'] = df['sibsp'] + df['parch'] + 1

print("📊 Familiengrößen-Verteilung:")
print(df['family_size'].value_counts().sort_index())

In [ ]:
# Erstelle 'is_alone' Feature
df['is_alone'] = (df['family_size'] == 1).astype(int)

print(f"📊 Alleinreisende Passagiere: {df['is_alone'].sum()}")

In [ ]:
# Analysiere Überleben nach Familiengröße
family_survival = df.groupby('family_size')['survived'].mean()
fig = px.bar(x=family_survival.index, y=family_survival.values,
             title='Überlebensrate nach Familiengröße',
             labels={'x': 'Familiengröße', 'y': 'Überlebensrate'})
fig.update_layout(yaxis_tickformat='.0%')
fig.show()

---
---
---

# 🏠 WIEDERHOLUNG & VERTIEFUNG (Für zu Hause oder Nachbereitung)

---
## Wiederholungsblock 1: Strukturierte vs. Unstrukturierte Daten (10 mins)

### 📝 Übung W1.1: Datentypen identifizieren - SOLUTION

In [ ]:
# Identifiziere verschiedene Datentypen im Datensatz
print("📊 Datentypen:")
print(df.dtypes)

# Trenne numerische und kategorische Features
numerical_cols = df.select_dtypes(include=[np.number]).columns.tolist()
categorical_cols = df.select_dtypes(include=['object', 'category']).columns.tolist()

print(f"\n✅ Numerische Features: {len(numerical_cols)}")
print(numerical_cols)
print(f"\n✅ Kategorische Features: {len(categorical_cols)}")
print(categorical_cols)

---
## Wiederholungsblock 2: Erweiterte Imputation-Strategien (20 mins)

### 📝 Übung W2.1: Gruppenbasierte Imputation für Alter - SOLUTION

In [ ]:
# Fülle fehlende Alterswerte mit dem Median der gleichen Klasse und Geschlecht
df['age_group_imputed'] = df.groupby(['pclass', 'sex'])['age'].transform(lambda x: x.fillna(x.median()))

print("✅ Gruppenbasierte Imputation abgeschlossen!")
print("\n📊 Median-Alter nach Klasse und Geschlecht:")
print(df.groupby(['pclass', 'sex'])['age'].median())

### 📝 Übung W2.2: Cabin-Feature Engineering - SOLUTION

In [ ]:
# Erstelle ein binäres Feature 'cabin_known' (1 wenn Cabin bekannt, sonst 0)
df['cabin_known'] = df['cabin'].notna().astype(int)

# Prüfe Korrelation mit Überleben
print("📊 Überlebensrate nach Cabin-Information:")
print(df.groupby('cabin_known')['survived'].mean())

print(f"\n📊 Fehlende Cabin-Werte: {df['cabin'].isnull().sum()} ({df['cabin'].isnull().mean()*100:.1f}%)")

---
## Wiederholungsblock 3: Ausreißer-Erkennung mit IQR (20 mins)

### 📝 Übung W3.1: IQR-Methode für Fahrpreise - SOLUTION

In [ ]:
# Erkenne Ausreißer bei 'fare' mit der IQR-Methode
# IQR = Q3 - Q1
# Ausreißer: < Q1 - 1.5×IQR  oder  > Q3 + 1.5×IQR

Q1 = df['fare'].quantile(0.25)
Q3 = df['fare'].quantile(0.75)
IQR = Q3 - Q1

lower_bound = Q1 - 1.5 * IQR
upper_bound = Q3 + 1.5 * IQR

print(f"📊 Q1 (25%): {Q1:.2f}")
print(f"📊 Q3 (75%): {Q3:.2f}")
print(f"📊 IQR: {IQR:.2f}")
print(f"📊 Untere Grenze: {lower_bound:.2f}")
print(f"📊 Obere Grenze: {upper_bound:.2f}")

# Zähle Ausreißer
outliers = df[(df['fare'] < lower_bound) | (df['fare'] > upper_bound)]
print(f"\n⚠️ Anzahl Ausreißer: {len(outliers)}")

---
## Wiederholungsblock 4: One-Hot Encoding (15 mins)

### 📝 Übung W4.1: Einschiffungshafen kodieren - SOLUTION

In [ ]:
# Wende One-Hot Encoding auf 'embarked' an
embarked_encoded = pd.get_dummies(df['embarked'], prefix='embarked')

print("📊 Original 'embarked' Spalte:")
print(df['embarked'].value_counts())
print("\n📊 One-Hot Encoded:")
print(embarked_encoded.head(10))

---
## Wiederholungsblock 5: Erweiterte Feature Engineering (25 mins)

### 📝 Übung W5.1: Titel aus Namen extrahieren - SOLUTION

In [ ]:
# Extrahiere Titel aus der 'name' Spalte (Mr., Mrs., Miss., etc.)
df['title'] = df['name'].str.extract(' ([A-Za-z]+)\.', expand=False)

print("📊 Gefundene Titel:")
print(df['title'].value_counts())

# Gruppiere seltene Titel
common_titles = ['Mr', 'Miss', 'Mrs', 'Master']
df['title_grouped'] = df['title'].apply(lambda x: x if x in common_titles else 'Other')

print("\n📊 Gruppierte Titel:")
print(df['title_grouped'].value_counts())

### 📝 Übung W5.2: Altersgruppen erstellen - SOLUTION

In [ ]:
# Erstelle Altersgruppen
# Kategorien: Kind (0-12), Jugendlich (13-19), Erwachsener (20-59), Senior (60+)

bins = [0, 12, 19, 59, 100]
labels = ['Kind', 'Jugendlich', 'Erwachsener', 'Senior']
df['age_group'] = pd.cut(df['age'], bins=bins, labels=labels, right=True)

print("📊 Altersgruppen-Verteilung:")
print(df['age_group'].value_counts().sort_index())

# Analysiere Überleben nach Altersgruppe
print("\n📊 Überlebensrate nach Altersgruppe:")
print(df.groupby('age_group')['survived'].mean())

### 📝 Übung W5.3: Eigenes Feature erstellen! - SOLUTION

**Challenge:** Erstelle ein eigenes Feature, das für die Überlebensvorhersage nützlich sein könnte!

In [ ]:
# Eigenes Feature erstellen: Fahrpreis pro Person
# Idee: Fahrpreis pro Person (fare / family_size)
df['fare_per_person'] = df['fare'] / df['family_size']

print("📊 Fahrpreis pro Person - Statistiken:")
print(df['fare_per_person'].describe())

# Analysiere wie das Feature mit Überleben korreliert
print("\n📊 Überlebensrate nach Fahrpreis-Kategorie:")
df['fare_category'] = pd.qcut(df['fare_per_person'], q=4, labels=['Niedrig', 'Mittel', 'Hoch', 'Sehr Hoch'], duplicates='drop')
print(df.groupby('fare_category')['survived'].mean())

---
## 🎯 Zusammenfassung & Reflexion

### Haupterkenntnisse von Tag 4:

✅ **Wissenspyramide:** Daten → Information → Wissen → Weisheit  
✅ **Fehlende Daten:** Mean, Median, Gruppenbasierte Imputation  
✅ **Ausreißer-Erkennung:** IQR-Methode, Box Plots  
✅ **Normalisierung:** Min-Max Scaling, Standardization  
✅ **Kategorische Kodierung:** One-Hot Encoding, Binary Encoding  
✅ **Feature Engineering:** Family Size, Title, Age Groups, Cabin Known

### Wichtig fürs Projekt:
- **Datenqualität prüfen** ist der erste Schritt!
- **Feature Engineering** kann die Modellleistung stark verbessern
- **Ausreißer** nicht immer entfernen - sie können wichtige Informationen enthalten!

### Reflexionsfragen:

1. **Welche Imputation-Strategie würdest du für das Abschlussprojekt wählen? Warum?**
   
   ___________________________________

2. **Welches neue Feature war am interessantesten/nützlichsten?**
   
   ___________________________________

3. **Wie könntest du Feature Engineering in deinem eigenen Projekt anwenden?**
   
   ___________________________________

---
## 📚 Ressourcen & Weiterführendes Material

- **Scikit-learn Preprocessing:** https://scikit-learn.org/stable/modules/preprocessing.html
- **Missing Data Handling:** https://pandas.pydata.org/docs/user_guide/missing_data.html
- **Feature Engineering Guide:** https://www.kaggle.com/learn/feature-engineering

---
**Nächste Session: Tag 6 - Machine Learning!** 🤖